In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
df=pd.read_csv('query1.csv')

In [3]:
df1=pd.read_csv('query2.csv')

In [4]:
result = pd.merge(df,df1[['Date', 'Blocks', 'Output_Satoshis']],on='Date')
result.head()

,Date,Transactions,Blocks,Output_Satoshis
0,2016-08-12,149,571788,192449213278677
1,2016-08-13,163,537632,204909291864336
2,2016-08-14,163,533825,171656943644686
3,2016-08-15,146,502906,207713022974418
4,2016-08-16,135,610164,273195830621631


In [5]:
result.describe()

,Transactions,Blocks,Output_Satoshis
count,760.000000,7.600000e+02,7.600000e+02
mean,152.388158,6.563773e+05,2.014011e+14
std,17.481648,1.457100e+05,1.057827e+14
min,2.000000,1.548900e+04,1.922581e+12
25%,142.000000,5.596760e+05,1.278973e+14
50%,153.000000,6.378340e+05,1.853705e+14
75%,164.000000,7.279940e+05,2.484635e+14
max,197.000000,1.264802e+06,9.838137e+14


In [6]:
import plotly.express as px
import plotly.offline as pyo
fig = px.line(result, x='Date', y='Transactions')
fig.show()

ModuleNotFoundError: No module named 'plotly'

In [ ]:
import plotly.express as px
fig = px.line(result, x='Date', y='Blocks')
fig.show()

In [ ]:
px.scatter_3d(result, x='Date', y='Transactions',z='Blocks')

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import IsolationForest 

data = result[['Output_Satoshis','Blocks','Transactions']]
outliers_fraction=0.05
scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)

# train isolation forest
model =  IsolationForest(contamination=outliers_fraction)
model.fit(data)

In [ ]:
y_pred = model.fit_predict(data)

In [ ]:
y_pred

In [ ]:
result['anomaly_IsolationForest'] = pd.Series(model.predict(data))


In [ ]:
result

In [ ]:
result['anomaly_IsolationForest'] = pd.Series(model.predict(data))
result['anomaly_IsolationForest'] = result['anomaly_IsolationForest'].apply(lambda x: x == -1)
result['anomaly_IsolationForest'] = result['anomaly_IsolationForest'].astype(int)


In [7]:
result

,Date,Transactions,Blocks,Output_Satoshis
0,2016-08-12,149,571788,192449213278677
1,2016-08-13,163,537632,204909291864336
2,2016-08-14,163,533825,171656943644686
3,2016-08-15,146,502906,207713022974418
4,2016-08-16,135,610164,273195830621631
...,...,...,...,...
755,2018-09-06,150,645864,127295710140994
756,2018-09-07,152,670653,108604821016277
757,2018-09-08,150,531346,71886241720552
758,2018-09-09,163,495661,72006661552204


In [12]:
a = result.loc[result['anomaly_IsolationForest'] == 1]

KeyError: 'anomaly_IsolationForest'

In [9]:
a.index

NameError: name 'a' is not defined

In [10]:
px.scatter_3d(result, x='Output_Satoshis', y='Transactions',z='Blocks')

NameError: name 'px' is not defined

In [11]:
px.scatter_3d(result,x='Output_Satoshis',y='Transactions',z='anomaly_IsolationForest')

NameError: name 'px' is not defined

In [25]:
X = result.iloc[:, 1:4].values
y = result.iloc[:, 4].values

In [35]:
X.sha

(760, 3)

In [34]:
y.shape

(760,)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [28]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [36]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'relu', input_dim = 3))

# Adding the second hidden layer
classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

W1119 22:48:58.515160 140058581796672 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/100
608/608 [==============================] - 2s 3ms/step - loss: 0.6765 - acc: 0.9523
Epoch 2/100
608/608 [==============================] - 0s 147us/step - loss: 0.6237 - acc: 0.9523
Epoch 3/100
608/608 [==============================] - 0s 137us/step - loss: 0.5322 - acc: 0.9523
Epoch 4/100
608/608 [==============================] - 0s 131us/step - loss: 0.4121 - acc: 0.9523
Epoch 5/100
608/608 [==============================] - 0s 176us/step - loss: 0.3020 - acc: 0.9523
Epoch 6/100
608/608 [==============================] - 0s 159us/step - loss: 0.2264 - acc: 0.9523
Epoch 7/100
608/608 [==============================] - 0s 139us/step - loss: 0.1827 - acc: 0.9523
Epoch 8/100
608/608 [==============================] - 0s 131us/step - loss: 0.1585 - acc: 0.9523
Epoch 9/100
608/608 [==============================] - 0s 144us/step - loss: 0.1444 - acc: 0.9523
Epoch 10/100
608/608 [==============================] - 0s 141us/step - loss: 0.1352 - acc: 0.9523
Epoch 11/100
608/608 